In [19]:
import sys
import pandas as pd
import json
import os
import pickle
import re
import difflib
from difflib import SequenceMatcher
import jellyfish
pd.set_option('display.max_columns', 100)

In [20]:
import spotipy
import spotipy.util as util

In [21]:
user_name = "Pillsbury Darboy"
client_id = "bff1afed3b63465aa6cdba9ec40f6afd"
client_secret = "53634fc86e8a4f71b7459197084e3cdf"
redirect_uri = "http://localhost/"
scope = 'user-library-read'
from spotipy.util import prompt_for_user_token
token = prompt_for_user_token(username=user_name,scope=scope,client_id=client_id,client_secret=client_secret,redirect_uri=redirect_uri)
from spotipy import Spotify
spotify = Spotify(auth=token)

In [22]:
class SetlistGenerator_Spotify(object):

    def __init__(self, seed_tracks='', seed_genres='', seed_artists=''):
        # build initial seed genres, tracks, and artists upon creation
        self.seed_genres = seed_genres
        self.seed_tracks = seed_tracks
        self.seed_artists = seed_artists
        
        # necessary in order to query from the API
        self.AuthorizeSpotify()
        
        #Setlist starts with seed_track
        self.set_list = self.get_track_metadata_single()
        
        self.genres = ['chicago-house', 'club', 'dancehall', 'deep-house', 'detroit-techno', 'dub', 'edm',
         'electro', 'electronic', 'groove', 'house', 'idm', 'industrial', 'minimal-techno', 'progressive-house', 
          'soul', 'techno', 'trance']
        self.moods = ['happy', 'sad']
        
        self.track_pool = self.getRecommendations()
        
    def getRecommendations(self):
        # Query spotify for recommendations based on a track
        recommendations = self.spotify.recommendations(seed_artists=self.seed_artists, 
                                                       seed_genres=self.seed_genres, 
                                                       seed_tracks=self.seed_tracks, 
                                                       limit = 50)
        # Convert spotify data into readable format
        metadata = self.get_track_metadata(recommendations)
        
        # Retrieve audio features for spotify data
        audio_features = pd.DataFrame(self.spotify.audio_features(self.get_track_ids(recommendations)))
        df = metadata.merge(audio_features)
        
        #Correct key + mode data 
        df = self.key_mode_convert(df)
        return(df)

    def getNextTrack(self, method = 0, min_dance=0.6):
        candidates = self.track_pool
        current_song_key = int(self.set_list[-1:]['camelot'])
        if method == 0:
            candidates = self.filter_by_camelot(df = candidates, 
                                                song_key_camelot = current_song_key)
            candidates = self.filter_by_popularity(candidates)
            candidates = self.filter_by_danceability(candidates, min_dance=min_dance)
        else:
            pass
        self.set_list = self.set_list.append(candidates.sample(1))
        self.set_list = self.set_list.reset_index(drop=True)
        
        return(self.set_list)
    
    ############################################
    ######      SEED     UTILITIES        ######
    ############################################
    
    def changeSeedTracks(self, use_last = 2):
        self.seed_tracks = list(self.set_list['id'][-use_last:])
        
    def changeSeedGenres(self, new_genres):
        self.seed_genres = new_genres
        
    def changeSeedArtists(self, new_artists):
        self.seed_genres = artists
        
    ############################################
    ######    NARRATIVE     UTILITIES     ######
    ############################################
    
    def CreateTurnarounds(self, **kwargs):
        for key, value in kwargs.items():
            print(key, value)
        self.narrative_valence  = np.random()
        self.narrative_energy  = np.random()
        self.narrative_mode  = np.random()
        self.narrative_tempo  = np.random()
        self.narrative_popularity  = np.random()
        self.narrative_danceability  = np.random()
        self.narrative_genre = np.random()
        
    def CreateNarratives(self, **kwargs):
        for key, value in kwargs.items():
            print(key, value)
        self.narrative_valence  = np.random()
        self.narrative_energy  = np.random()
        self.narrative_mode  = np.random()
        self.narrative_tempo  = np.random()
        self.narrative_popularity  = np.random()
        self.narrative_danceability  = np.random()
        self.narrative_genre = np.random()
        
    def NarrativeUtilDanceability(self):
        ## Mode 1 ##
        # Seed track is climax track.
        pass
    
    def NarrativeUtilMode(self):
        pass
    
    def NarrativeUtilPopularity(self):
        pass
    
    def NarrativeUtilEnergy(self):
        pass
    
    def NarrativeUtilValence(self):
        pass
    
    ############################################
    ######     TRACK   POOL   UTILITIES   ######
    ############################################
        
    def getTrackPool(self):
        return self.track_pool
    
    def expandTrackPool(self):
        self.track_pool = self.track_pool.append(self.getRecommendations())
        self.cleanTrackPool()
        
        print('Expanded track pool')
    
    def cleanTrackPool(self):
        self.track_pool = self.track_pool[-self.track_pool['id'].isin(list(self.set_list['id']))]
        self.track_pool = self.track_pool.drop_duplicates()
        self.track_pool = self.track_pool.reset_index(drop=True)

        
    ############################################
    ######    TRACK SELECTION UTILITIES   ######
    ############################################
        
    def closest_keys(self, camelot):
        closest = []
        closest.append(camelot)
        closest.append((camelot + 1)%12)
        closest.append((camelot - 1)%12)
        return(closest)
    
    def filter_by_camelot(self, df, song_key_camelot):
        filtered = []
        for i in self.closest_keys(song_key_camelot):
            filtered.append(pd.DataFrame(df[df['camelot'] == i]))
        return(pd.concat(filtered))
        
    def filter_by_popularity(self, df, min_pop=0, max_pop=100):
        df = df[df['popularity'] >= min_pop]
        df = df[df['popularity'] <= max_pop]
        return(df)
    
    def filter_by_tempo(self, df, min_tempo = 90, max_tempo = 145):
        df = df[df['tempo'] >= min_tempo]
        df = df[df['tempo'] <= max_tempo]
        return(df)
    
    def filter_by_valence(self, df=None, min_valence = 0, max_valence = 1):
        df = self.track_pool
        df = df[df['valence'] >= min_valence]
        df = df[df['valence'] <= max_valence]
        return(df)
    
    def filter_by_danceability(self, df=None, min_dance = 0, max_dance = 1):
        df = self.track_pool
        df = df[df['danceability'] >= min_dance]
        df = df[df['danceability'] <= max_dance]
        return(df)
    
    def filter_by_energy(self, df=None, min_energy = 0, max_energy = 1):
        df = self.track_pool
        df = df[df['energy'] >= min_energy]
        df = df[df['energy'] <= max_energy]
        return(df)
    
    def filter_by_liveness(self, df=None, min_liveness = 0, max_liveness = 1):
        df = self.track_pool
        df = df[df['liveness'] >= min_liveness]
        df = df[df['liveness'] <= max_liveness]
        return(df)
    
    def filter_by_acousticness(self, df=None, min_acoustic = 0, max_acoustic = 1):
        df = self.track_pool
        df = df[df['acousticness'] >= min_acoustic]
        df = df[df['acousticness'] <= max_acoustic]
        return(df)
    
    def filter_by_instrumentalness(self, df=None,
                                   min_instrumentalness = 0, 
                                   max_instrumentalness = 1):
        df = self.track_pool
        df = df[df['instrumentalness'] >= min_instrumentalness]
        df = df[df['instrumentalness'] <= max_instrumentalness]
        return(df)



    ############################################
    ######        METADATA UTILITIES      ######
    ############################################
    
    def get_track_metadata(self, recommendations):
        tracklist_frames = []
        for track in recommendations['tracks']:
            tracklist_frames.append({'track': track['name'],'artist': track['artists'][0]['name'],
                                     'duration':track['duration_ms'],
                                    'track_number': track['track_number'], 'id': track['id'], 
                                     'popularity':track['popularity'],
                                     'disc_number':track['disc_number'], 'album':track['album']['name']})
        return(pd.DataFrame(tracklist_frames))

    def get_track_metadata_single(self):
        track_id = self.seed_tracks[0]
        track = self.spotify.track(track_id)
        metadata = pd.DataFrame({'track': track['name'],'artist': track['artists'][0]['name'],
                                     'duration':track['duration_ms'],
                                    'track_number': track['track_number'], 'id': track['id'], 
                                     'popularity':track['popularity'],
                                     'disc_number':track['disc_number'], 'album':track['album']['name']}, index =[0])
        audio_features = pd.DataFrame(self.spotify.audio_features(track_id))
        df = metadata.merge(audio_features)
        df = self.key_mode_convert(df)
        return(df)
    
    def get_track_ids(self, recommendations):
        id_list = []
        for spotify_id in recommendations['tracks']:
            id_list.append(spotify_id['id'])
        return(id_list)
    
    def key_mode_convert(self, recommendations):
        df = recommendations
        camelot_list = []
        for i in range(0, len(df)):
            if df['mode'][i] == 1:
                if (df['key'][i] == 0):
                    camelot_list.append(7)
                if (df['key'][i] == 1):
                    camelot_list.append(2)
                if (df['key'][i] == 2):
                    camelot_list.append(9)
                if (df['key'][i] == 3):
                    camelot_list.append(4)
                if (df['key'][i] == 4):
                    camelot_list.append(11)                
                if (df['key'][i] == 5):
                    camelot_list.append(6)                
                if (df['key'][i] == 6):
                    camelot_list.append(1)                
                if (df['key'][i] == 7):
                    camelot_list.append(8)                
                if (df['key'][i] == 8):
                    camelot_list.append(3)                
                if (df['key'][i] == 9):
                    camelot_list.append(10)                
                if (df['key'][i] == 10):
                    camelot_list.append(5)
                if (df['key'][i] == 11):
                    camelot_list.append(0)
            else:
                if (df['key'][i] == 0):
                    camelot_list.append(4)
                if (df['key'][i] == 1):
                    camelot_list.append(11)
                if (df['key'][i] == 2):
                    camelot_list.append(6)
                if (df['key'][i] == 3):
                    camelot_list.append(1)
                if (df['key'][i] == 4):
                    camelot_list.append(8)                
                if (df['key'][i] == 5):
                    camelot_list.append(3)                
                if (df['key'][i] == 6):
                    camelot_list.append(10)                
                if (df['key'][i] == 7):
                    camelot_list.append(5)                
                if (df['key'][i] == 8):
                    camelot_list.append(0)                
                if (df['key'][i] == 9):
                    camelot_list.append(7)                
                if (df['key'][i] == 10):
                    camelot_list.append(2)
                if (df['key'][i] == 11):
                    camelot_list.append(9)
        df['camelot'] = camelot_list
        return(df)

    ############################################
    ######        SPOTIFY  UTILITIES      ######
    ############################################
    
    def SearchSpotify(self, search_string):
        return self.spotify.search(search_string)
    

    
    def AuthorizeSpotify(self):
        print('Authorizing Spotify')
        self.user_name = "Ronald Marp"
        self.client_id = '1e05e910ef664f27949005267fdd0936'
        self.client_secret ='7e65fb89a86a4a9c849924f5bd95661a'
        self.redirect_uri = "http://localhost/"
        self.scope = 'user-library-read'
        self.token = prompt_for_user_token(username=self.user_name,scope=self.scope,
                                           client_id=self.client_id,
                                           client_secret=self.client_secret,
                                           redirect_uri=self.redirect_uri)
        self.spotify = Spotify(auth=self.token)

In [23]:
# Some example tracks to start with
## Percolator Meme - Teeth ['2QT5afoBT6nQabNuvcxXUa']

In [29]:
seed_artist_id = search_for_artistID('Vril Vortekz')


Found artist Vril


In [28]:
seed_track_id = search_for_trackID('Vril Vortekz')

Found track Vortekz by Vril


In [24]:
def search_for_artistID(search_string):
    result = spotify.search(str(search_string))['tracks']['items'][0]['artists'][0]
    print('Found artist %s' %result['name'])
    return(result['id'])

In [25]:
def search_for_trackID(search_string):
    result = spotify.search(str(search_string))['tracks']['items'][0]
    print('Found track %s by %s' %(result['name'], result['artists'][0]['name']))
    return(result['id'])

In [14]:
seed_artist_id = search_for_artistID('Aleksi Perala')

Found artist Aleksi Perälä


In [15]:
seed_track_id = search_for_trackID('Triadz Hubot')

Found track Triadz - Original Mix by Hubot


In [30]:
d = SetlistGenerator_Spotify(seed_tracks=[seed_track_id], seed_artists=[seed_artist_id])
                             #seed_artists=[seed_artist_id], 
                             #seed_genres=['minimal-techno', 'techno']) #seed_genres=['minimal-techno', 'techno', 'detroit-techno'])

Authorizing Spotify


In [31]:
def GenerateSetList(setlist_generator, setlength = 10, methods = 1):
    d = setlist_generator
    while len(d.set_list) <= setlength:
        d.getNextTrack()
        d.changeSeedTracks()
        d.expandTrackPool()
    return(d)
    

In [32]:
while len(d.set_list) < 10:
    d.getNextTrack()
    d.changeSeedTracks()
    d.expandTrackPool()

d.set_list

Expanded track pool
Expanded track pool
Expanded track pool
Expanded track pool
Expanded track pool
Expanded track pool
Expanded track pool
Expanded track pool
Expanded track pool


,album,artist,disc_number,duration,id,popularity,track,track_number,acousticness,analysis_url,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,track_href,type,uri,valence,camelot
0,Vortekz,Vril,1,320193,1w8cgxMCxfhrLkOEWGWWiJ,20,Vortekz,1,0.041100,https://api.spotify.com/v1/audio-analysis/1w8c...,0.823,320193,0.950,0.753,9,0.0683,-6.451,0,0.0941,123.469,4,https://api.spotify.com/v1/tracks/1w8cgxMCxfhr...,audio_features,spotify:track:1w8cgxMCxfhrLkOEWGWWiJ,0.465,7
1,MDRL2,Roman Lindau,1,392127,0lA0gOfLLPDbs3ypYWwrCr,0,HECK,1,0.001750,https://api.spotify.com/v1/audio-analysis/0lA0...,0.798,392127,0.885,0.753,11,0.0679,-8.110,0,0.1440,126.008,4,https://api.spotify.com/v1/tracks/0lA0gOfLLPDb...,audio_features,spotify:track:0lA0gOfLLPDbs3ypYWwrCr,0.512,9
2,Talis / Into The Red,Peter Van Hoesen,1,500857,2eZYhuwMD2SwQcxsxmY8TN,16,Talis,1,0.179000,https://api.spotify.com/v1/audio-analysis/2eZY...,0.788,500857,0.829,0.674,11,0.0836,-9.355,1,0.0578,124.971,4,https://api.spotify.com/v1/tracks/2eZYhuwMD2Sw...,audio_features,spotify:track:2eZYhuwMD2SwQcxsxmY8TN,0.777,0
3,Realm Of Consciousness,Monoloc,1,368732,0cPuHIBSAyIP7zhUYLYGMl,26,Phoenix,2,0.436000,https://api.spotify.com/v1/audio-analysis/0cPu...,0.685,368732,0.437,0.906,6,0.1380,-15.596,0,0.0734,124.000,4,https://api.spotify.com/v1/tracks/0cPuHIBSAyIP...,audio_features,spotify:track:0cPuHIBSAyIP7zhUYLYGMl,0.070,10
4,Analog Passion,Luke Hess,1,414706,4p0ZAHHt6mbNqFg1zGaMUT,6,Paskho Version 1,1,0.047700,https://api.spotify.com/v1/audio-analysis/4p0Z...,0.801,414706,0.782,0.866,10,0.1190,-11.017,0,0.0721,125.012,4,https://api.spotify.com/v1/tracks/4p0ZAHHt6mbN...,audio_features,spotify:track:4p0ZAHHt6mbNqFg1zGaMUT,0.348,2
5,Lichtbedingt,Mike Dehnert,1,270275,4IL8XBMjtLI6X4Oq1Mtyv1,5,En Outre,12,0.006810,https://api.spotify.com/v1/audio-analysis/4IL8...,0.776,270276,0.713,0.758,1,0.1290,-8.088,1,0.0678,119.017,4,https://api.spotify.com/v1/tracks/4IL8XBMjtLI6...,audio_features,spotify:track:4IL8XBMjtLI6X4Oq1Mtyv1,0.278,2
6,Zero,Subjected,1,352328,31fHyB2Granyqy1rstDGrG,25,Concept 3,5,0.000522,https://api.spotify.com/v1/audio-analysis/31fH...,0.776,352328,0.665,0.567,11,0.1130,-7.613,0,0.2270,128.041,4,https://api.spotify.com/v1/tracks/31fHyB2Grany...,audio_features,spotify:track:31fHyB2Granyqy1rstDGrG,0.158,9
7,Fourth Method,Ancient Methods,1,389680,5w3U6VGfrPliQrEWicAPOu,9,AM-4 A1,1,0.129000,https://api.spotify.com/v1/audio-analysis/5w3U...,0.769,389680,0.762,0.828,10,0.1830,-11.187,0,0.0851,124.025,4,https://api.spotify.com/v1/tracks/5w3U6VGfrPli...,audio_features,spotify:track:5w3U6VGfrPliQrEWicAPOu,0.254,2
8,Perc Trax - Tracks Of 2014,Exium,1,323210,2tWQaekLpevzo7GeTvHr3i,4,Raw Visions,8,0.000712,https://api.spotify.com/v1/audio-analysis/2tWQ...,0.678,323210,0.876,0.834,1,0.1150,-8.061,1,0.0770,130.002,4,https://api.spotify.com/v1/tracks/2tWQaekLpevz...,audio_features,spotify:track:2tWQaekLpevzo7GeTvHr3i,0.218,2
9,Coast to Coast,Peter Van Hoesen,1,360809,3hgxqf2CddtMb9eDrlDGeg,2,Voodoo Benny's Wishbone,3,0.041300,https://api.spotify.com/v1/audio-analysis/3hgx...,0.773,360810,0.977,0.858,2,0.1150,-7.554,1,0.0812,124.016,4,https://api.spotify.com/v1/tracks/3hgxqf2CddtM...,audio_features,spotify:track:3hgxqf2CddtMb9eDrlDGeg,0.277,9


In [ ]:
# Danceability Narrative Prototype
d = SetlistGenerator_Spotify(seed_tracks=[seed_track_id], seed_artists=[seed_artist_id], seed_genres=['techno'])
# First Expand the track pool to allow for more flexibility in narrative development
d.expandTrackPool()
d.expandTrackPool()
d.changeSeedGenres(['ambient', 'minimal-techno'])
d.expandTrackPool()
d.expandTrackPool()
d.changeSeedGenres(['ambient', 'techno'])
d.expandTrackPool()
d.expandTrackPool()
d.changeSeedGenres(['ambient'])
d.expandTrackPool()
d.expandTrackPool()
d.changeSeedGenres(['techno'])
while len(d.set_list) < 10:
    d.getNextTrack()
    d.changeSeedTracks()
    d.expandTrackPool()

d.set_list

In [39]:
d.set_list['uri'].to_csv('../../../../spotify-downloader/output.txt', index=False)

In [34]:
spotify.recommendation_genre_seeds()

{'genres': ['acoustic',
  'afrobeat',
  'alt-rock',
  'alternative',
  'ambient',
  'anime',
  'black-metal',
  'bluegrass',
  'blues',
  'bossanova',
  'brazil',
  'breakbeat',
  'british',
  'cantopop',
  'chicago-house',
  'children',
  'chill',
  'classical',
  'club',
  'comedy',
  'country',
  'dance',
  'dancehall',
  'death-metal',
  'deep-house',
  'detroit-techno',
  'disco',
  'disney',
  'drum-and-bass',
  'dub',
  'dubstep',
  'edm',
  'electro',
  'electronic',
  'emo',
  'folk',
  'forro',
  'french',
  'funk',
  'garage',
  'german',
  'gospel',
  'goth',
  'grindcore',
  'groove',
  'grunge',
  'guitar',
  'happy',
  'hard-rock',
  'hardcore',
  'hardstyle',
  'heavy-metal',
  'hip-hop',
  'holidays',
  'honky-tonk',
  'house',
  'idm',
  'indian',
  'indie',
  'indie-pop',
  'industrial',
  'iranian',
  'j-dance',
  'j-idol',
  'j-pop',
  'j-rock',
  'jazz',
  'k-pop',
  'kids',
  'latin',
  'latino',
  'malay',
  'mandopop',
  'metal',
  'metal-misc',
  'metalcore',


In [ ]:
len(mixesdb)

In [ ]:
# Load and Process Datasets for big-data generation

spotify = pd.read_csv('data/spotify/spotify_database_mergedV2.csv')
del spotify['Unnamed: 0']
del spotify['Unnamed: 0.1']
spotify['artist_and_track'] = spotify['artist'] + ' ' + spotify['track']
print("Read and processed Spotify data")

discogs_m = pd.read_csv('data/discogs/discogs_mixed.csv')
discogs_r = pd.read_csv('data/discogs/discogs_releases.csv')
to_concat = [discogs_m, discogs_r]
discogs = pd.concat(to_concat)
del discogs_m
del discogs_r
del discogs['Unnamed: 0']
del discogs['genre']
del discogs['labels']
del discogs['length']
discogs['artist'] = discogs['artist'].apply(lambda x: re.sub(pattern='\([0-9]*\)', string=x, repl = '').strip())
discogs['artist_and_track'] = discogs['artist'] + ' ' + discogs['track']
print("Read and processed Discogs data")

mixesdb = pd.read_csv('data/mixesdb/mixesdb_all_sets_featurized.csv')
del mixesdb['Unnamed: 0']
mixesdb['artist_and_track'] = mixesdb['artist'] + ' ' + mixesdb['track']
print("Read and processed MixesDB data")


In [ ]:
spotify['popularity'].max()